# Imports

In [33]:
!pip install -U tensorflow==2.6.4
!pip install cloud-tpu-client

In [34]:
!pip install -qq /kaggle/input/keras-cv-attention-models/keras_cv_attention_models-1.3.9-py3-none-any.whl

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydicom
import os
import cv2
import tensorflow as tf
from cloud_tpu_client import Client
from tensorflow import keras
import tensorflow_addons as tfa
from keras_cv_attention_models import convnext
from tensorflow.keras import backend as K
from google.cloud import storage

In [36]:
tf.version.VERSION

'2.6.4'

In [37]:
from cloud_tpu_client import Client
try:
    Client().configure_tpu_version(tf.__version__, restart_type='ifNeeded')
except ValueError as e:
    print(e)

# Define GCS Storage Area

In [38]:
client_area = 'kagglersna01'
storage_client = storage.Client(project=client_area)

# Delete All

In [39]:
def delete_file(filepath):
    print('deleting ' + filepath + ' from local')
    os.remove('/kaggle/working/' + filepath)
    
def clear_all_local():
    for k in os.listdir('/kaggle/working/'):
        if k == '.virtual_documents':
            continue
        delete_file(k)

In [40]:
import shutil

In [41]:
def clear_all():
    shutil.rmtree('/kaggle/working/checkpoints')
    clear_all_local()
#clear_all()

# Init TPU

In [42]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470


2023-02-20 16:34:45.237385: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2023-02-20 16:34:45.237475: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:32788}
2023-02-20 16:34:45.243417: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2023-02-20 16:34:45.243469: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:32788}


REPLICAS:  8


# Setup Params

In [43]:
# GCS_DS_PATH = KaggleDatasets().get_gcs_path('rsna-preprocessing-tfrecords-640x512-dataset-pub')

imsize = [int(1344/2), int(768/2), 1] 
epoch = 50
batchsize = 8 * strategy.num_replicas_in_sync

trim = 43764 
validim = 10942 
step_per_epoch = (trim // batchsize)
steps_per_val_epoch = (validim // batchsize) + 1

wd = 0.0001
lr = 5e-6 * strategy.num_replicas_in_sync
cyc = 0.50

# Parsing Function

In [44]:
def parse_example(tfrecord):
    feature_desc = {
        'image': tf.io.FixedLenFeature([], tf.string, default_value=""),
        'label': tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        'age': tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        'impant': tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        'laterality': tf.io.FixedLenFeature([], tf.string, default_value=""),
        'view': tf.io.FixedLenFeature([], tf.string, default_value=""),
        'diff_neg': tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }
    
    example = tf.io.parse_single_example(tfrecord, feature_desc)
    image = tf.io.decode_jpeg(example["image"], channels=1)
    image = tf.reshape(image, shape=imsize)
    return image, tf.cast(example["label"], tf.float32)

# Data Augmentations

> **INPUT** - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3] 

> **OUTPUT** - image with CT squares of side size SZ*DIM removed

In [45]:
def dropout(image, dim, PROBABILITY=0.6, CT=5, SZ=0.10):
    
    # DO DROPOUT WITH PROBABILITY DEFINED ABOVE
    P = tf.cast( tf.random.uniform([], 0, 1)<PROBABILITY, tf.int32)
    if (P==0) | (CT==0) | (SZ==0): 
        return image
    
    for k in range(CT):
        
        # CHOOSE RANDOM LOCATION
        x = tf.cast(tf.random.uniform([], 0, dim[0]),tf.int32)
        y = tf.cast(tf.random.uniform([], 0, dim[1]),tf.int32)
        
        # COMPUTE SQUARE 
        WIDTH = tf.cast(SZ*DIM, tf.int32) * P
        ya = tf.math.maximum(0, y-WIDTH//2)
        yb = tf.math.minimum(DIM, y+WIDTH//2)
        xa = tf.math.maximum(0, x-WIDTH//2)
        xb = tf.math.minimum(DIM, x+WIDTH//2)
        
        # DROPOUT IMAGE
        one = image[ya:yb, 0:xa, :]
        two = tf.zeros([yb-ya, xb-xa, 3], dtype=image.dtype) 
        three = image[ya:yb, xb:DIM, :]
        middle = tf.concat([one, two, three], axis=1)
        image = tf.concat([image[0:ya,:,:], middle, image[yb:DIM, :, :]], axis=0)
        image = tf.reshape(image, [DIM, DIM, 3])

    return image

In [46]:
data_augm_lay = keras.Sequential(
    [
        keras.layers.RandomZoom(height_factor=(0, -0.3))
    ]
)

In [47]:
def augment_image(image, y):
    
    image = tf.image.random_brightness(image, 0.10)
    # image = tf.image.random_contrast(image, 0.90, 1.40)
    # image = tf.image.random_saturation(image, 0.50, 2.00)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_jpeg_quality(image, 75, 100)
    
    image = data_augm_lay(image)
    # image = dropout(image)
    
    return image, y

In [48]:
sample_per = 1 - (2.0770496298327394 / 10)

In [49]:
def undersample_majority(x, y):
    return y == 1 or tf.random.uniform([]) > sample_per

# Get Dataset From GCS Files

In [50]:
def record_dataset(filepaths, shuffle_buffer_size=5000, batch_size=32, training=True, ordered=False):
    
    ignore_order = tf.data.Options()
    
    if not ordered:
        ignore_order.experimental_deterministic = False
    
    dataset = tf.data.TFRecordDataset(filepaths, num_parallel_reads=tf.data.AUTOTUNE)
    
    # dataset = dataset.cache()
    
    dataset = dataset.map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    
    if training:
        dataset = dataset.filter(undersample_majority)
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.with_options(ignore_order)
        # dataset = dataset.shuffle(shuffle_buffer_size)
        dataset = dataset.repeat()
        
    dataset = dataset.batch(batch_size)
    
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [51]:
trainpaths = ['gs://' + 'train_batches_smaller' + '/' + "{}_batch_{:0>3}.tfrecord".format('train', index) for index in range(0, int(200/2))]
validpaths = ['gs://' + 'train_batches_smaller' + '/' + "{}_batch_{:0>3}.tfrecord".format('valid', index) for index in range(0, int(50/2))]

train_set = record_dataset(trainpaths, batch_size=batchsize)
valid_set = record_dataset(validpaths, batch_size=batchsize, training=False)

# Metric/s

In [52]:
class pFBeta(tf.keras.metrics.Metric):
    def __init__(self, beta=1, name='pF1', **kwargs):
        super().__init__(name=name, **kwargs)
        self.beta = beta
        self.epsilon = 1e-10
        self.pos = self.add_weight(name='pos', initializer='zeros')
        self.ctp = self.add_weight(name='ctp', initializer='zeros')
        self.cfp = self.add_weight(name='cfp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 0, 1)
        pos = tf.cast(tf.reduce_sum(y_true), tf.float32)
        ctp = tf.cast(tf.reduce_sum(y_pred[y_true == 1]), tf.float32)
        cfp = tf.cast(tf.reduce_sum(y_pred[y_true == 0]), tf.float32)
        self.pos.assign_add(pos)
        self.ctp.assign_add(ctp)
        self.cfp.assign_add(cfp)

    def result(self):
        beta2 = self.beta * self.beta
        prec = self.ctp / (self.ctp + self.cfp + self.epsilon)
        reca = self.ctp / (self.pos + self.epsilon)
        return (1 + beta2) * prec * reca / (beta2 * prec + reca)

    def reset_state(self):
        self.pos.assign(0.)
        self.ctp.assign(0.)
        self.cfp.assign(0.)

# Build Model

In [53]:
def norm(image):
    image = tf.repeat(image, repeats=3, axis=3)
    image = tf.cast(image, tf.float32)
    image = tf.keras.applications.imagenet_utils.preprocess_input(image, mode='tf')
    return image

In [54]:
# print(os.listdir('./model/'))

In [55]:
from_saved = False
model_name = 'resnet101-V4.h5'

In [56]:
def get_utility():
    metrics = [
        pFBeta(name='pF1'),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.BinaryAccuracy()
    ]
    
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    optimizer = tf.optimizers.Adam(learning_rate=lr)
    
    return metrics, loss, optimizer

In [57]:
if from_saved:    
    with strategy.scope():
        load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
        model = tf.keras.models.load_model(
            './model/' + model_name, 
            options=load_locally, 
            custom_objects={
                "ConvUnit": ConvUnit, 
                "IdentityUnit": IdentityUnit, 
                'pFBeta': pFBeta,
                'pF1_thr': pfbeta_thr
            }
        )
        
        metrics, loss, optimizer = get_utility()
        
else:
    with strategy.scope():
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=imsize, dtype=tf.uint8))
        model.add(keras.layers.Lambda(lambda x: norm(x)))
        model.add(convnext.ConvNeXtV2Tiny(
            input_shape=[int(1344/2), int(768/2), 3],
            pretrained='imagenet21k-ft1k',
            num_classes=0))
        model.add(keras.layers.GlobalAvgPool2D())
        model.add(keras.layers.Dropout(0.30))
        model.add(keras.layers.Dense(1, activation='sigmoid'))

        metrics, loss, optimizer = get_utility()

>>>> Load pretrained from: /root/.keras/models/convnext_v2_tiny_384_imagenet21k-ft1k.h5


In [58]:
def lr_fn(step, epochs=epoch, warmup=0, lr_max=lr, cycle=cyc):
    if step < warmup:
        lr = lr_max * 0.10 ** (warmup - step)
    else:
        progress = float(step - warmup) / float(max(1, epochs - warmup))
        lr = max(0.0, 0.5 * (1.0 + np.cos(2.0 * np.pi * float(cycle) * progress))) * lr_max
    return lr

lr_schedule = [lr_fn(step) for step in range(epoch)]
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda step: lr_schedule[step], verbose=0)

In [59]:
class WeightDecayCallback(tf.keras.callbacks.Callback):
    def __init__(self, wd_ratio=wd):
        self.step_counter = 0
        self.wd_ratio = wd_ratio
    
    def on_epoch_begin(self, epoch, logs=None):
        model.optimizer.weight_decay = model.optimizer.learning_rate * self.wd_ratio
        print(f"Learning rate: {model.optimizer.learning_rate.numpy():.2e}")
        print(f"Weight decay: {model.optimizer.weight_decay.numpy():.2e}")
        
wd_callback = WeightDecayCallback()

In [60]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

In [61]:
tf.keras.backend.clear_session()

tf.config.optimizer.set_jit(True)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 672, 384, None)    0         
_________________________________________________________________
convnext_v2_tiny (Functional (None, 21, 12, 768)       27864960  
_________________________________________________________________
global_average_pooling2d (Gl (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 27,865,729
Trainable params: 27,865,729
Non-trainable params: 0
_________________________________________________________________


# Train

In [62]:
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10, 
    restore_best_weights=True,
    mode='max',
    monitor='val_pF1'
)

history = model.fit(
    train_set, 
    steps_per_epoch=step_per_epoch,
    epochs=epoch, 
    validation_data=valid_set,
    validation_steps=steps_per_val_epoch,
    class_weight = {
        0: 1.0,
        1: 10.0 
    },
    callbacks=[early_stopping_cb, lr_callback, wd_callback]
)

Epoch 1/50
Learning rate: 4.00e-05
Weight decay: 4.00e-09
683/683 [==============================] - 378s 407ms/step - loss: 1.2385 - pF1: 0.1841 - precision: 0.1309 - recall: 0.5847 - binary_accuracy: 0.6048 - val_loss: 1.0164 - val_pF1: 0.0555 - val_precision: 0.0306 - val_recall: 0.8876 - val_binary_accuracy: 0.3568
Epoch 2/50
Learning rate: 4.00e-05
Weight decay: 4.00e-09
683/683 [==============================] - 291s 426ms/step - loss: 1.0282 - pF1: 0.2441 - precision: 0.2073 - recall: 0.6920 - binary_accuracy: 0.7237 - val_loss: 0.6925 - val_pF1: 0.0682 - val_precision: 0.0427 - val_recall: 0.7068 - val_binary_accuracy: 0.6331
Epoch 3/50
Learning rate: 3.98e-05
Weight decay: 3.98e-09
683/683 [==============================] - 262s 384ms/step - loss: 0.9081 - pF1: 0.2798 - precision: 0.2452 - recall: 0.7314 - binary_accuracy: 0.7703 - val_loss: 0.3194 - val_pF1: 0.0956 - val_precision: 0.0811 - val_recall: 0.4217 - val_binary_accuracy: 0.8781
Epoch 4/50
Learning rate: 3.96e-05
We

In [63]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model.save('./model/convex-smaller-V2.h5', options=save_locally)